# Load Images

In [1]:
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from google.cloud import storage
from google.oauth2 import service_account

In [3]:
def get_images_to_folder():
    BUCKET_NAME = "youtube_optimizer_data"
    storage_dir = "thumbnail_images"
    local_dir = "bucket_data/" #Create this manually

    my_credentials = service_account.Credentials.from_service_account_file("massive-pen-365111-8eaed18fb748.json")

    client = storage.Client(credentials=my_credentials)
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(storage_dir)
    
    blobs = bucket.list_blobs(prefix =storage_dir)
    for blob in blobs:
        filename = blob.name.replace('/','_')
        blob.download_to_filename(local_dir + filename)

In [4]:
len('thumbnail_images_')

17

In [5]:
def load_images_from_folder(folder='/Users/nicolafriedrich/code/jacksharples1/youtube_optimizer/bucket_data'):
    images = []
    views = []
    for filename in os.listdir(folder):
        if len(filename)< len('thumbnail_images_')+1:
            continue

        else:
            img = plt.imread(os.path.join(folder,filename))
            if img.shape != (180,320,3):
                print(img.shape)
                continue
            else:
                last_underscore = filename.rfind('_')
                y = int(filename[last_underscore +1:])
                images.append(list(img))
                views.append(y)
    return images, views


In [6]:
#imgs,views = load_images_from_folder()

In [7]:
#np.array(imgs).shape

In [8]:
#len(imgs),imgs[0].shape,type(imgs),type(imgs[0])

In [9]:
#def split_train_test(X,y,test_size = 0.3, random_state = 0):
#    return train_test_split(X, y, test_size=0.3, random_state=0)

In [25]:
def dataloading():
    get_images_to_folder()
    X,y = load_images_from_folder()
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test

In [26]:
X_train, X_test, y_train, y_test = dataloading()

(90, 120, 3)
(90, 120, 3)
(90, 120, 3)


In [27]:
X_train.shape

(441, 180, 320, 3)

In [13]:
#np.unique(np.array([array.shape for array in X_train]), axis=0, return_counts=True)

In [14]:
#X_train[0].shape, X_train[193].shape

In [15]:
#X_train = np.concatenate(X_train, axis=0)

In [16]:
#X_train[0][0]

# Model

In [17]:
from keras.applications.xception import Xception
from keras.layers import GlobalAveragePooling2D, Dense, Flatten
from keras.models import Sequential
#from keras.optimizers import SGD
#import numpy as np

2022-11-29 16:52:20.999286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def base_model():
    base_model = Xception(weights="imagenet",input_shape = (180,320,3),include_top=False)
    base_model.trainable = False
    return base_model

In [19]:
def complete_model():
    model = Sequential((
        base_model(),
        GlobalAveragePooling2D(),
        Flatten(),
        Dense(50,activation = 'relu'),
        Dense(1,activation = 'linear')))
    
    model.compile(loss="mse", optimizer='adam',
                  metrics=["mae"])
    
    return model  

In [20]:
model = complete_model()
model.summary()

2022-11-29 16:52:29.805221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 6, 10, 2048)       20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 50)                102450    
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 20,963,981
Trainable params: 102,501
Non-trainable params: 20,861,480
______________________________________

In [21]:
#X_train  = np.array(X_train)
#y_train = np.array(y_train)

In [22]:
len(y_train)

441

In [28]:
type(y_train)

numpy.ndarray

In [29]:
history = model.fit(X_train,y_train, epochs = 5,validation_split=0.2)

Epoch 1/5
11/11 [==============================] - 98s 9s/step - loss: 2541832512733184.0000 - mae: 11020399.0000 - val_loss: 2722584600772608.0000 - val_mae: 14613054.0000
Epoch 2/5
11/11 [==============================] - 84s 8s/step - loss: 2541829291507712.0000 - mae: 11020216.0000 - val_loss: 2722578426757120.0000 - val_mae: 14612857.0000
Epoch 3/5
11/11 [==============================] - 86s 8s/step - loss: 2541825533411328.0000 - mae: 11020050.0000 - val_loss: 2722571715870720.0000 - val_mae: 14612653.0000
Epoch 4/5
11/11 [==============================] - 73s 7s/step - loss: 2541820433137664.0000 - mae: 11019891.0000 - val_loss: 2722564736548864.0000 - val_mae: 14612454.0000
Epoch 5/5
11/11 [==============================] - 77s 7s/step - loss: 2541815601299456.0000 - mae: 11019690.0000 - val_loss: 2722556683485184.0000 - val_mae: 14612217.0000


# Baseline model

In [30]:
from sklearn.metrics import mean_absolute_error

In [31]:
np.mean(np.abs(y_train - np.mean(y_train)))

18506997.37747132